In [175]:
#必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#モデルの分割
from sklearn.model_selection import train_test_split
#正規化
from sklearn.preprocessing import StandardScaler
# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [176]:
#CSVファイルの読み込み
dataset = pd.read_csv("atp_tennis.csv")
# datasetの型を確認
type(dataset)

pandas.core.frame.DataFrame

In [177]:
# 最初の5行を表示
dataset.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,63,77,-1,-1,-1.0,-1.0,6-4 6-2
1,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Clement A.,Enqvist T.,Enqvist T.,56,5,-1,-1,-1.0,-1.0,3-6 3-6
2,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,Escude N.,40,655,-1,-1,-1.0,-1.0,6-7 7-5 6-3
3,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Knippschild J.,Federer R.,Federer R.,87,65,-1,-1,-1.0,-1.0,1-6 4-6
4,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,Fromberg R.,81,198,-1,-1,-1.0,-1.0,7-6 5-7 6-4


In [178]:
# データの形状を確認
dataset.shape
# 全体的な情報を表示
dataset.info()
# 列ラベルを確認
dataset.columns.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66681 entries, 0 to 66680
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tournament  66681 non-null  object 
 1   Date        66681 non-null  object 
 2   Series      66681 non-null  object 
 3   Court       66681 non-null  object 
 4   Surface     66681 non-null  object 
 5   Round       66681 non-null  object 
 6   Best of     66681 non-null  int64  
 7   Player_1    66681 non-null  object 
 8   Player_2    66681 non-null  object 
 9   Winner      66681 non-null  object 
 10  Rank_1      66681 non-null  int64  
 11  Rank_2      66681 non-null  int64  
 12  Pts_1       66681 non-null  int64  
 13  Pts_2       66681 non-null  int64  
 14  Odd_1       66681 non-null  float64
 15  Odd_2       66681 non-null  float64
 16  Score       66681 non-null  object 
dtypes: float64(2), int64(5), object(10)
memory usage: 8.6+ MB


array(['Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round',
       'Best of', 'Player_1', 'Player_2', 'Winner', 'Rank_1', 'Rank_2',
       'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2', 'Score'], dtype=object)

In [179]:
# 要約統計量を表示
dataset.describe()

,Best of,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2
count,66681.000000,66681.000000,66681.000000,66681.000000,66681.000000,66681.000000,66681.000000
mean,3.377529,75.846103,75.543543,1134.708973,1139.502527,2.414427,2.411957
std,0.782649,100.610092,100.879448,1721.139941,1744.762999,2.660011,2.640809
min,3.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,3.000000,24.000000,24.000000,117.000000,120.000000,1.333000,1.333000
50%,3.000000,53.000000,54.000000,701.000000,701.000000,1.736667,1.800000
75%,3.000000,92.000000,92.000000,1250.000000,1253.000000,2.750000,2.750000
max,5.000000,3390.000000,4915.000000,16950.000000,16950.000000,67.000000,51.000000


In [180]:
# データ準備
dataset = dataset[(dataset['Rank_1'] > 0) & (dataset['Rank_2'] > 0) & (dataset['Odd_1'] > 0) & (dataset['Odd_2'] > 0)]
dataset['target'] = (dataset['Winner'] == dataset['Player_1']).astype(int)

feature_cols = ['Rank_1', 'Rank_2', 'Pts_1', 'Pts_2', 'Odd_1', 'Odd_2']
X = dataset[feature_cols]
y = dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [181]:
# ※データの正規化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [182]:
# モデル構築
model = Sequential()

# 入力層
model.add(Dense(16, activation='relu', input_shape=(6,)))

# 隠れ層
model.add(Dense(8, activation='relu'))

# 出力層（2択の問題=1, sigmoid ）
model.add(Dense(1, activation='sigmoid'))

c:\Users\ocean\Documents\aidev\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [183]:
#学習前の設定（keras）
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])


In [184]:
# 学習の実施
history = model.fit(X_train, y_train, # 変数名を y_train
                    epochs=5000,batch_size=32,verbose=1,
                    validation_split=0.2, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=100, verbose=1)], )

# 評価
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'正解率: {accuracy * 100:.2f}%')


Epoch 1/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 2s 847us/step - accuracy: 0.6900 - loss: 0.5818 - val_accuracy: 0.6886 - val_loss: 0.5797
Epoch 2/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - accuracy: 0.6971 - loss: 0.5726 - val_accuracy: 0.6893 - val_loss: 0.5763
Epoch 3/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 789us/step - accuracy: 0.6972 - loss: 0.5718 - val_accuracy: 0.6910 - val_loss: 0.5773
Epoch 4/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.6967 - loss: 0.5713 - val_accuracy: 0.6899 - val_loss: 0.5762
Epoch 5/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 789us/step - accuracy: 0.6967 - loss: 0.5713 - val_accuracy: 0.6903 - val_loss: 0.5771
Epoch 6/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 792us/step - accuracy: 0.6969 - loss: 0.5711 - val_accuracy: 0.6903 - val_loss: 0.5762
Epoch 7/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.6972 - loss: 0.5708 - val_accuracy: 0.6905 - val_loss: 0.5761
Epoch 8/5000
1258/1258 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - ac